In [2]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from openai import OpenAI  # Changed import
import httpx
from dotenv import load_dotenv
import os


In [3]:
# Initialize embeddings and vectorstore (unchanged)
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = Chroma(
    persist_directory="../job_profiles_db2",
    embedding_function=embeddings,
    collection_name="job_profiles"
)

In [4]:
# Retrieve context documents (unchanged)
query = "Which job profiles have administration related skills?"
context_docs = vectorstore.similarity_search(query, k=3)
context = "\n\n".join([doc.page_content for doc in context_docs])

In [5]:
PROMPT_TEMPLATE = f"""System: You're an expert in answering questions about a library of job profiles. Use this context:
{context}

User: {query}
Assistant: """

In [10]:
import requests
r = requests.get('http://localhost:8000/v1/models')
r.json()

{'object': 'list',
 'data': [{'id': 'job-profile-rag',
   'object': 'model',
   'created': 1739562390,
   'owned_by': 'your-company',
   'capabilities': {'completions': True, 'chat_completions': True}}]}

In [ ]:
# Initialize OpenAI client with local endpoint
import socket
server = f'{socket.gethostname()}.local'

client = OpenAI(
    base_url="http://localhost:1234/v1",  # Use IPv4 loopback
    api_key="None",  # Can be any string if your local endpoint doesn't require auth
)

# Create and send request using OpenAI format
response = client.chat.completions.create(
    model="deepseek-r1-distill-qwen-1.5b",  # Model name expected by your local endpoint
    messages=[
        {"role": "system", "content": "You're an expert in answering questions about a library of job profiles."},
        {"role": "user", "content": f"Context: {context}\n\nQuestion: {query}"}
    ],
    max_tokens=300,
    temperature=0.7
)

# Process response
if response and response.choices:
    print("Response:", response.choices[0].message.content)
else:
    print("No response received")

Response: All three job profiles have administration related skills.

The Training Administrator coordinates financial and administrative processes for a training program, enters training data into HRIS and training database, and runs reports on learning activities.

The Program Assistant Supervisory provides office administrative, secretarial, and financial support services, and coordinates the day-to-day priorities of the manager.

Industrial Relations Officers (IROs) also have administrative responsibilities, such as writing investigation reports consistent with legal requirements and conducting representation votes for the Labour Relations Board. Additionally, they mentor and train Employment Standards Officers on investigative techniques and best practices for investigations, which involves administrative tasks related to training and development.
